In [11]:
import os
import librosa
import pickle
from tqdm import tqdm
import numpy as np

In [2]:
data_dir = '/data4/TTS/VCTK/VCTK-Corpus/'

In [3]:
with open('/data4/TTS/VCTK/VCTK-Corpus/txt/p225/p225_006.txt','r') as fp:
    print(fp.readlines())

['When the sunlight strikes raindrops in the air, they act as a prism and form a rainbow.\n']


In [4]:
y, sr = librosa.load('/data4/TTS/VCTK/VCTK-Corpus/wav48/p225/p225_006.wav', sr=16000)
print(y, sr)

[ 0.00019856  0.00056936  0.00056342 ... -0.00718605 -0.00768058
  0.        ] 16000


In [6]:
dataset = {
    'page' : list(),
    'index' : list(),
    'audio_path' : list(),
    'audio_array' : list(),
    'text' : list(),
    'sample_rate' : 16000
}

for page in os.listdir(os.path.join(data_dir, 'txt')):
    print(page)
    for fname in os.listdir(os.path.join(data_dir, 'txt', page)):
        audio_path = os.path.join(data_dir, 'wav48', page, fname[:-4] + '.wav')
        text_path = os.path.join(data_dir, 'txt', page, fname)
        dataset['page'].append(page)
        dataset['index'].append(fname[:-4])
        dataset['audio_path'].append(audio_path)
        dataset['audio_array'].append(librosa.load(audio_path, sr=dataset['sample_rate'])[0])
        with open(text_path) as fp:
            text = ' '.join(fp.readlines()).replace('\n','')
            dataset['text'].append(text)
            
with open(os.path.join(data_dir, 'dataset-vctk-16k.pkl'), 'wb') as f:
    pickle.dump(dataset, f)

p311
p263
p251
p326
p374
p340
p286
p246
p247
p280
p302
p233
p228
p335
p237
p254
p287
p336
p298
p343
p317
p273
p257
p347
p281
p278
p360
p285
p236
p271
p245
p310
p312
p238
p297
p303
p351
p250
p318
p274
p330
p341
p306
p305
p255
p288
p259
p275
p230
p227
p276
p272
p314
p268
p253
p339
p334
p260
p231
p248
p345
p249
p295
p243
p258
p277
p252
p362
p256
p229
p284
p225
p376
p232
p262
p333
p270
p239
p267
p282
p269
p261
p234
p329
p266
p364
p363
p294
p292
p307
p293
p313
p300
p299
p361
p264
p240
p226
p241
p308
p279
p304
p323
p301
p265
p283
p244
p316


In [ ]:
with open('/data4/TTS/VCTK-Corpus/dataset-vctk-16k.pkl', 'rb') as f:
    dataset = pickle.load(f)
del dataset['page'], dataset['index'], dataset['audio_path']


max_audio_length = 0
for arr in dataset['audio_array']:
    if len(arr) > max_audio_length:
        max_audio_length = len(arr)
print(max_audio_length)

In [8]:
from transformers import Wav2Vec2FeatureExtractor

cache_dir = "/data4/yoomcache"
model_cache_dir = os.path.join(cache_dir, 'huggingface')
wav2vec_pretrained = "facebook/wav2vec2-base"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(wav2vec_pretrained, 
                                                             cache_dir=model_cache_dir)




for idx in tqdm(range(len(dataset['audio_array']))):
    dataset['audio_array'][idx] = feature_extractor(dataset['audio_array'][idx], 
                                                    sampling_rate=dataset['sample_rate'],
                                                    padding='max_length',
                                                    max_length=max_audio_length
                                                    ).input_values[0]

100%|████████████████████████████████████| 44070/44070 [01:21<00:00, 539.38it/s]


In [12]:
dataset['audio_array'] = np.stack(dataset['audio_array'])

In [13]:
dataset.keys()

dict_keys(['audio_array', 'text', 'sample_rate'])

In [14]:
with open('/data4/TTS/VCTK-Corpus/dataset-vctk-16k (numpy).pkl', 'wb') as f:
    pickle.dump(dataset, f)